In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("data.csv")
data.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44,1,28.0,0,> 2 Years,Yes,40454.0,26.0,217,1
1,2,Male,76,1,3.0,0,1-2 Year,No,33536.0,26.0,183,0
2,3,Male,47,1,28.0,0,> 2 Years,Yes,38294.0,26.0,27,1
3,4,Male,21,1,11.0,1,< 1 Year,No,28619.0,152.0,203,0
4,5,Female,29,1,41.0,1,< 1 Year,No,27496.0,152.0,39,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   Gender                381109 non-null  object 
 2   Age                   381109 non-null  int64  
 3   Driving_License       381109 non-null  int64  
 4   Region_Code           381109 non-null  float64
 5   Previously_Insured    381109 non-null  int64  
 6   Vehicle_Age           381109 non-null  object 
 7   Vehicle_Damage        381109 non-null  object 
 8   Annual_Premium        381109 non-null  float64
 9   Policy_Sales_Channel  381109 non-null  float64
 10  Vintage               381109 non-null  int64  
 11  Response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


In [4]:
data.isna().sum()

id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
Response                0
dtype: int64

In [5]:
data.dropna(inplace=True)

In [6]:
data.drop(columns=["Policy_Sales_Channel", "Vintage"],inplace=True)

In [7]:
data["Region_Code"]= data["Region_Code"].astype('int')
data

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Response
0,1,Male,44,1,28,0,> 2 Years,Yes,40454.0,1
1,2,Male,76,1,3,0,1-2 Year,No,33536.0,0
2,3,Male,47,1,28,0,> 2 Years,Yes,38294.0,1
3,4,Male,21,1,11,1,< 1 Year,No,28619.0,0
4,5,Female,29,1,41,1,< 1 Year,No,27496.0,0
...,...,...,...,...,...,...,...,...,...,...
381104,381105,Male,74,1,26,1,1-2 Year,No,30170.0,0
381105,381106,Male,30,1,37,1,< 1 Year,No,40016.0,0
381106,381107,Male,21,1,30,1,< 1 Year,No,35118.0,0
381107,381108,Female,68,1,14,0,> 2 Years,Yes,44617.0,0


In [8]:
data[["Vehicle_Age_lt_1_year","Vehicle_Age_gt_1_year"]] = pd.get_dummies(data.Vehicle_Age, drop_first=True,dtype='int')


In [9]:
data.drop(columns=["Vehicle_Age","id"],inplace=True)


In [11]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categorical_features = ["Gender", "Vehicle_Damage"]
numeric_features = ["Age","Driving_License","Region_Code","Previously_Insured","Annual_Premium","Vehicle_Age_lt_1_year","Vehicle_Age_gt_1_year"]

In [12]:
from sklearn.ensemble import RandomForestClassifier

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(), categorical_features),
        ('num', "passthrough",numeric_features)
    ]
)

pipeline = Pipeline(steps=[
    ("preprocesser", preprocessor),
    ("classifier", RandomForestClassifier(random_state=42))
])

In [ ]:
from sklearn.model_selection import train_test_split


X = data.drop(columns=["Response"])
y = data["Response"]

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=42,test_size=.2)


In [19]:
pipeline.fit(X_train,y_train)

,steps,"[('preprocesser', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [20]:
y_pred = pipeline.predict(X_test)

from sklearn.metrics import classification_report

print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91     66699
           1       0.33      0.23      0.27      9523

    accuracy                           0.85     76222
   macro avg       0.61      0.58      0.59     76222
weighted avg       0.82      0.85      0.83     76222



In [26]:
import pickle

filename = "pipeline.pkl"

with open(filename,"wb") as f:
    pickle.dump(pipeline, f)

In [30]:
with open(filename,"rb") as f:
    model = pickle.load(f)

In [32]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,model.predict(X_test)))

0.8450447377397602


In [33]:
data

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Response,Vehicle_Age_lt_1_year,Vehicle_Age_gt_1_year
0,Male,44,1,28,0,Yes,40454.0,1,0,1
1,Male,76,1,3,0,No,33536.0,0,0,0
2,Male,47,1,28,0,Yes,38294.0,1,0,1
3,Male,21,1,11,1,No,28619.0,0,1,0
4,Female,29,1,41,1,No,27496.0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
381104,Male,74,1,26,1,No,30170.0,0,0,0
381105,Male,30,1,37,1,No,40016.0,0,1,0
381106,Male,21,1,30,1,No,35118.0,0,1,0
381107,Female,68,1,14,0,Yes,44617.0,0,0,1


In [36]:
X_train

,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Damage,Annual_Premium,Vehicle_Age_lt_1_year,Vehicle_Age_gt_1_year
332803,Female,39,1,15,0,Yes,52906.0,0,0
116248,Male,38,1,11,0,Yes,23038.0,0,0
255005,Male,22,1,30,1,No,45318.0,1,0
317474,Female,23,1,41,1,No,29132.0,1,0
344212,Male,56,1,48,0,Yes,2630.0,0,1
...,...,...,...,...,...,...,...,...,...
259178,Female,24,1,36,1,No,22575.0,1,0
365838,Male,56,1,35,0,Yes,41287.0,0,0
131932,Female,22,1,2,0,Yes,18857.0,1,0
146867,Male,44,1,32,1,Yes,2630.0,0,0
